In [1]:
import numpy as np
import cv2
import scipy
from scipy.spatial.distance import cdist
import random
import sys
import os
import glob
import matplotlib.pyplot as plt

dirname = r'..\ubdata'
files = [x for x in os.listdir(dirname) if x.endswith(".jpg")]
print(files)
print(dirname)



def match2images(img1,img2):
    #print("entered match2")
    sift =  cv2.xfeatures2d.SIFT_create()
    #sift=cv2.ORB_create()
    pair1_matches=[]
    pair2_matches=[]
    p1_1=[]
    p1_2=[]
    p2_1=[]
    p2_2=[]
    gray1=to_grayscale(img1)
    gray2=to_grayscale(img2)
    kp1,desc1 = sift.detectAndCompute(gray1,None)
    kp2,desc2 = sift.detectAndCompute(gray2,None)
    
    
    #img1=cv2.drawKeypoints(gray1,kp1,img1)
    #img2=cv2.drawKeypoints(gray2,kp2,img2)
    #cv2.imwrite('sift_keypoints.jpg',img1)
    #cv2.imwrite('sift_keypoints2.jpg',img2)
    #imgplot = plt.imshow(img1)
    #plt.show()
    #imgplot = plt.imshow(img2)
    #plt.show()
    pair1=scipy.spatial.distance.cdist(desc1,desc2,'seuclidean')
    thres1 = 3
    for i in range(0,pair1.shape[0]):
        for j in range(0,pair1.shape[1]):
            if pair1[i][j]<=thres1:
                pair1_matches.append((i,j))
    return len(pair1_matches)

def findDistance(p1,p2,H):
    pt1=[]
    pt1.append(p1[0])
    pt1.append(p1[1])
    pt1.append(1)
    pt2=[]
    pt2.append(p2[0])
    pt2.append(p2[1])
    pt2.append(1)
    
    pt1=np.transpose(np.matrix(pt1))
    temp = np.matmul(H, pt1)
    temp = (temp/temp.item(2))    
    pt2=np.transpose(np.matrix(pt2))
    dist= cv2.norm(pt2,temp,cv2.NORM_L2)
    return dist
def process_homo_data(s,t):
    s= np.asarray(s)
    t= np.asarray(t)
    return s.flatten(),t.flatten()

def to_grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   # if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break
def compute_homography(s, t):
        #print("enter homography")

        sf,tf=process_homo_data(s,t)
        cons = np.transpose(tf)
        s_x = sf[0::2]
        s_y = sf[1::2]    
        t_x = tf[0::2]
        t_y = tf[1::2]
        Coef = []
        for i in range(0, len(s_x)):
            Coef.append([s_x[i], s_y[i], 1, 0, 0, 0, ((-s_x[i]) * t_x[i]), ((-s_y[i]) * t_x[i])])
            Coef.append([0, 0, 0, s_x[i], s_y[i], 1, ((-s_x[i]) * t_y[i]), ((-s_y[i]) * t_y[i])])
        #print("done homography")
        A = np.array(Coef)
        x = np.linalg.lstsq(A, cons)[0]
        homo_matrix = np.array([[x[0], x[1], x[2]], [x[3], x[4], x[5]], [x[6], x[7], 1]])    
        return homo_matrix

def Ransac_algo(pxx,pyy,iterations):
    print("enter ransac")
    maxInliers = []
    for i in range(iterations):
        random_ptx=[]
        random_pty=[]
        for j in range(0,4):
            ra = random.randrange(0, len(pxx))
            random_ptx.append(pxx[ra])
            random_pty.append(pyy[ra])
        #print("enter homography in ransac")
        h = compute_homography(random_ptx, random_pty)
        #print(h)
        inliersx = []
        inliersy = []
        for i in range(0,len(pxx)):
            d = findDistance(pxx[i],pyy[i], h)
            #print("d",d)
            if d < 5:
                inliersx.append(pxx[i])
                inliersy.append(pyy[i])
        #print(len(inliersx),"inliers")
        if len(inliersx) > len(maxInliers):
            maxInliers = inliersx
            besthomo = h
    print("besthomo")         
    print (besthomo)    
    return besthomo

def stitch3(im1, im2, im3,h1,h2):
    l1,b1 = im1.shape[:2]
    p1 = np.float32([[0,0], [0,l1], [b1, l1], [b1,0]]).reshape(-1,1,2)
    
    l2,b2 = im2.shape[:2]
    p2 = np.float32([[0,0], [0,l2], [b2, l2], [b2,0]]).reshape(-1,1,2)
    p2_ = cv2.perspectiveTransform(p2,h1 )
    
    l3,b3 = im3.shape[:2]
    p3 = np.float32([[0,0], [0,l3], [b3, l3], [b3,0]]).reshape(-1,1,2)
    p3_ = cv2.perspectiveTransform(p3, h2 )
    
    points = np.concatenate((p1, p2_,p3_))
    [x1, y1] = np.int32(points.min(axis=0).ravel()-0.5 )
    [x2, y2] = np.int32(points.max(axis=0).ravel()+0.5 )

    t = [-x1, -y1]
    t_mat = np.array([[1, 0, t[0]], [0, 1, t[1]], [0,0,1]])
    left = cv2.warpPerspective(im1,t_mat.dot(h1),(x2 - x1, y2 - y1))
    right = cv2.warpPerspective(im3,t_mat.dot(h2),(x2 - x1, y2 - y1))
    final_img = left + right
    final_img[t[1]:l1+t[1],t[0]:b1+t[0]] = im2
    return final_img

def stitch2(im1, im2,h):
    l1,b1 = im1.shape[:2]
    p1 = np.float32([[0,0], [0,l1], [b1, l1], [b1,0]]).reshape(-1,1,2)
    
    l2,b2 = im2.shape[:2]
    p2 = np.float32([[0,0], [0,l2], [b2, l2], [b2,0]]).reshape(-1,1,2)
    p2_ = cv2.perspectiveTransform(p2,h )
    
    
    points = np.concatenate((p1, p2_))
    [x1, y1] = np.int32(points.min(axis=0).ravel()-0.5 )
    [x2, y2] = np.int32(points.max(axis=0).ravel()+0.5 )
    t = [-x1, -y1]
    t_mat = np.array([[1, 0, t[0]], [0, 1, t[1]], [0,0,1]])
    final_img = cv2.warpPerspective(im1,t_mat.dot(h),(x2 - x1, y2 - y1))
    final_img[t[1]:l1+t[1],t[0]:b1+t[0]] = im2
    return final_img


# In[20]:



def do3file(files,dirname):
    
    pair1_matches=[]
    pair2_matches=[]
    p1_1=[]
    p1_2=[]
    p2_1=[]
    p2_2=[]
    
    try:
        img1 = cv2.imread(files[0])
        img2 = cv2.imread(files[1])
        img3 = cv2.imread(files[2])
        img1= cv2.resize(img1,None,fx=0.3,fy=0.3)
    #imgplot = plt.imshow(img1)
    #plt.show()
        img2= cv2.resize(img2,None,fx=0.3,fy=0.3)
    #imgplot = plt.imshow(img2)
    #plt.show()
        img3= cv2.resize(img3,None,fx=0.3,fy=0.3)
    #imgplot = plt.imshow(img3)
    #plt.show()
    except Exception as e:
        print(str(e))

    #sift = cv2.xfeatures2d.SIFT_create()
    #print(sift)
    #sift = cv2.ORB_create()

    imlist = [match2images(img1,img2),match2images(img2,img3),match2images(img1,img3)]
    mini= min(imlist)
    idx = imlist.index(mini)
    #print (mini)
    #print (idx)
    if idx == 0:
        middle = img3
        left = img1
        right =img2
    if idx == 1:
        middle =img1
        left = img2
        right =img3
    if idx == 2:
        middle =img2
        left = img1
        right =img3

    middle_left = middle[0:middle.shape[0],0:middle.shape[1]//3]
    left_right = left[0:left.shape[0],(left.shape[1]-left.shape[1]//3):left.shape[1]]
    right_right = right[0:right.shape[0],(right.shape[1]-right.shape[1]//3):right.shape[1]]

    if  (match2images(middle_left, left_right)) < (match2images(middle_left, right_right)):
        tt = left
        left =right
        right =tt

    #cv2.imwrite("1eft.jpg",left)
    #cv2.imwrite("2ight.jpg",right)
    #cv2.imwrite("3iddle.jpg",middle)
    print ("done ordering")
    img1 = left
    img2 = middle
    img3 = right

    img1cp = img1.copy()
    #imgplot = plt.imshow(img1cp)
    #plt.show()
    img2cp = img2.copy()
    img3cp = img3.copy()

    gray1= to_grayscale(img1)
    #imshow(gray1)
    #imgplot = plt.imshow(gray1)
    #plt.show()
    #sift = cv2.xfeatures2d.SIFT_create()
    gray2= to_grayscale(img2)
    gray3= to_grayscale(img3)

    sift = cv2.xfeatures2d.SIFT_create()
    

    #SIFT features
    kp1,desc1 = sift.detectAndCompute(gray1,None)
    kp2,desc2 = sift.detectAndCompute(gray2,None)
    kp3,desc3 = sift.detectAndCompute(gray3,None)
    #img1=cv2.drawKeypoints(gray1,kp1,img1)
    #img2=cv2.drawKeypoints(gray2,kp2,img2)
    #img3=cv2.drawKeypoints(gray3,kp3,img3)
    #imgplot = plt.imshow(img1)
    #plt.show()
    #imgplot = plt.imshow(img2)
    #plt.show()
    #imgplot = plt.imshow(img3)
    #plt.show()
    #cv2.imwrite('sift_keypoints.jpg',img1)
    #cv2.imwrite('sift_keypoints.jpg',img2)
    #cv2.imwrite('sift_keypoints.jpg',img3)

    pair1=scipy.spatial.distance.cdist(desc1,desc2,'seuclidean')
    pair2=scipy.spatial.distance.cdist(desc2,desc3,'seuclidean')

    thres1 = np.amin(pair1)*2
    thres2 = np.amin(pair2)*2

    for i in range(0,pair1.shape[0]):
        for j in range(0,pair1.shape[1]):
            if pair1[i][j]<=thres1:
                pair1_matches.append((i,j))

    for x in range(0,pair2.shape[0]):
        for y in range(0,pair2.shape[1]):
            if pair2[x][y]<=thres2:
                pair2_matches.append((x,y))

    for i in range (0,len(pair1_matches)):
        p1_1.append(kp1[pair1_matches[i][0]].pt)
        p1_2.append(kp2[pair1_matches[i][1]].pt)

    for i in range (0,len(pair2_matches)):
        p2_1.append(kp2[pair2_matches[i][0]].pt)
        p2_2.append(kp3[pair2_matches[i][1]].pt)

    p1_1=np.asarray(p1_1)
    p1_2=np.asarray(p1_2)

    p2_1=np.asarray(p2_1)
    p2_2=np.asarray(p2_2)

    p1xx = np.transpose(p1_1)
    p1yy = np.transpose(p1_2)
    p2xx = np.transpose(p2_1)
    p2yy = np.transpose(p2_2)
    print ("done matching")
    #Homo1 = compute_homography(p1xx.T,p1yy.T)
    while(1):
        try:

            Homo1 = Ransac_algo(p1xx.T,p1yy.T,500)
           
            break
        except:
            pass

    while(1):
        try:

            Homo2 = Ransac_algo(p2xx.T,p2yy.T,500)
            #print (Homo2)
            Homo2 = np.linalg.inv(Homo2)
            break
        except:
            pass

    result = stitch3(img1cp, img2cp, img3cp,Homo1,Homo2)
    
    imgplot = plt.imshow(result)
    plt.show()

    cv2.imwrite(dirname+"panoramafinal.jpg",result)


# In[21]:



def do2file(files,dirname):
    pair1_matches=[]
    
    p1_1=[]
    p1_2=[]
    try:
        img1 = cv2.imread(files[0])
        img2 = cv2.imread(files[1])
        img1= cv2.resize(img1,None,fx=0.3,fy=0.3)
        img2= cv2.resize(img2,None,fx=0.3,fy=0.3)
    except Exception as e:
        print(str(e))
    
    sift = cv2.xfeatures2d.SIFT_create()
    #sift = cv2.ORB_create()
    img1_left = img1[0:img1.shape[0],0:img1.shape[1]//3]
    img1_right = img1[0:img1.shape[0],(img1.shape[1]-img1.shape[1]//3):img1.shape[1]]
    img2_left = img2[0:img2.shape[0],0:img2.shape[1]//3]
    img2_right = img2[0:img2.shape[0],(img2.shape[1]-img2.shape[1]//3):img2.shape[1]]

    if  (match2images(img1_left, img2_right)) > (match2images(img2_left, img1_right)):
        temp = img1
        img1 = img2
        img2 = temp
    
    print ("done ordering 2")
    
    img1cp = img1.copy()
    img2cp = img2.copy()
    
    gray1= to_grayscale(img1)
    gray2= to_grayscale(img2)
    
   

    #SIFT features
    kp1,desc1 = sift.detectAndCompute(gray1,None)
    kp2,desc2 = sift.detectAndCompute(gray2,None)
    #img1=cv2.drawKeypoints(gray1,kp1,img1)
    #img2=cv2.drawKeypoints(gray2,kp2,img2)
    #img3=cv2.drawKeypoints(gray3,kp3,img3)
    #imgplot = plt.imshow(img1)
    #plt.show()
    #imgplot = plt.imshow(img2)
    #plt.show()
    #imgplot = plt.imshow(img3)
    #plt.show()
    
    pair1=scipy.spatial.distance.cdist(desc1,desc2,'seuclidean')
    
    thres1 = np.amin(pair1)*2
    
    for i in range(0,pair1.shape[0]):
        for j in range(0,pair1.shape[1]):
            if pair1[i][j]<=thres1:
                pair1_matches.append((i,j))

    
    for i in range (0,len(pair1_matches)):
        p1_1.append(kp1[pair1_matches[i][0]].pt)
        p1_2.append(kp2[pair1_matches[i][1]].pt)

    
    p1_1=np.asarray(p1_1)
    p1_2=np.asarray(p1_2)


    p1xx = np.transpose(p1_1)
    p1yy = np.transpose(p1_2)
    
    
    while(1):
        try:

            Homo1 = Ransac_algo(p1xx.T,p1yy.T,2000)
            print (Homo1)
            #Homo1 = np.linalg.inv(Homo1)
            break
        except:
            pass
        
    result = stitch2(img1cp, img2cp, Homo1)

    cv2.imwrite(dirname+"panorama.jpg",result)

    


if len(files) == 3:
    do3file(files,dirname)
    print("do3")
elif len(files) == 2:
    do2file(files,dirname)
    print("do2")
elif len(files)==1:
    cv2.imwrite(dirname+"panorama.jpg",files[0])
else:
    print ("Sorry No images or more than three images are given")

# In[22]:







['img1.jpg', 'img2.jpg', 'img3.jpg']
C:\Users\Chluc\Downloads\cvip\imgs
done ordering
done matching
enter ransac


C:\Users\Chluc\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


besthomo
[[ 2.17849401e+00 -2.94606324e-01 -1.25228714e+03]
 [ 1.00012014e+00  2.17120593e+00 -9.22978983e+02]
 [ 1.40303117e-03  4.12721214e-05  1.00000000e+00]]
enter ransac
besthomo
[[ 1.57894279e+00 -7.73409475e-02 -5.32143312e+02]
 [ 3.76043025e-01  1.48221491e+00 -3.14083175e+02]
 [ 5.09420539e-04  1.07316474e-04  1.00000000e+00]]


<Figure size 640x480 with 1 Axes>

do3
